Clasificarea țesuturilor cancerigene

Se consideră informații despre cancerul de sân la femei, informații extrase din ecografii mamare (detalii aici) precum: - Tipul malformației identificate (țesut benign sau țesut malign) - Caracteristici numerice ale nucleului celulelor din aceste țesuturi: - raza (media distanțelor între centru si punctele de pe contur) - textura (măsurată prin deviația standard a nivelelor de gri din imaginea asociată țesutului analizat) Folosindu-se aceste date, să se decidă dacă țesutul dintr-o nouă ecografie (pentru care se cunosc cele 2 caracteristici numerice – raza și textura –) va fi etichetat ca fiind malign sau benign.


Specificaţi, proiectaţi, implementaţi si testati cate un algoritm de clasificare pentru problema 2 si problema 3 bazat pe regresie logistica. Antrenati cate un clasificator pentru fiecare problema, pe care apoi sa ii utilizati pentru a stabili:

- daca o leziune (dintr-o mamografie) caracterizata printr-o textura de valoare 10 si o raza de valoare 18 este leziune maligna sau benigna


🏵️ Cerinte opționale

Rezolvarea unei probleme de regresie/clasificare prin:

- folosirea validarii încrucișate
- investigarea diferitelor funcții de loss
ce se întîmplă în cazul clasificarii binare daca se modifică pragul de decizie din 0.5 în alte valori. Cum se poate aprecia calitatea clasificatorului pentru diferite valori ale pragului?

In [3]:
import pandas as pd

column_names = [
    'id', 'diagnosis',
    'mean_radius', 'mean_texture', 'mean_perimeter', 'mean_area', 'mean_smoothness',
    'mean_compactness', 'mean_concavity', 'mean_concave_points', 'mean_symmetry', 'mean_fractal_dimension',
    'se_radius', 'se_texture', 'se_perimeter', 'se_area', 'se_smoothness',
    'se_compactness', 'se_concavity', 'se_concave_points', 'se_symmetry', 'se_fractal_dimension',
    'worst_radius', 'worst_texture', 'worst_perimeter', 'worst_area', 'worst_smoothness',
    'worst_compactness', 'worst_concavity', 'worst_concave_points', 'worst_symmetry', 'worst_fractal_dimension'
]

data = pd.read_csv("data/breast_cancer_wisconsin.csv", header=None, names=column_names)

data.to_csv("breast_cancer_wisconsin.csv", index=False)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [5]:

url = "data/wdbc.data"
column_names = ['id', 'diagnosis', 'radius', 'texture'] + [f'feature_{i}' for i in range(28)]
data = pd.read_csv(url, header=None, names=column_names)

feature1 = 'radius'
feature2 = 'texture'
inputs = data[[feature1, feature2]].values
outputs = data['diagnosis'].map({'M': 1, 'B': 0}).values

trainInputs, testInputs, trainOutputs, testOutputs = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

scaler = StandardScaler()
trainInputs = scaler.fit_transform(trainInputs)
testInputs = scaler.transform(testInputs)

In [6]:
def logistic_regression_tool():

    model = LogisticRegression()
    model.fit(trainInputs, trainOutputs)

    w0 = model.intercept_[0]
    w1, w2 = model.coef_[0]
    print(f'Model: P(Malign) = 1 / (1 + exp(-({w0:.4f} + {w1:.4f}*x1 + {w2:.4f}*x2)))')

    trainPred = model.predict(trainInputs)
    testPred = model.predict(testInputs)
    print(f'Accuracy - Train: {accuracy_score(trainOutputs, trainPred):.4f}')
    print(f'Accuracy - Test: {accuracy_score(testOutputs, testPred):.4f}')

    # Predictie pentru (18, 10)
    new_sample = scaler.transform([[18, 10]])
    prob = model.predict_proba(new_sample)[0][1]
    print(f'Probabilitate malign pentru (18,10): {prob:.4f} => {"Malign" if prob > 0.5 else "Benign"}')


logistic_regression_tool()

Model: P(Malign) = 1 / (1 + exp(-(-0.7168 + 3.1051*x1 + 0.8916*x2)))
Accuracy - Train: 0.8879
Accuracy - Test: 0.9035
Probabilitate malign pentru (18,10): 0.6845 => Malign


In [7]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            linear = np.dot(X, self.weights) + self.bias
            predictions = self._sigmoid(linear)

            dw = (1 / n_samples) * np.dot(X.T, (predictions - y))
            db = (1 / n_samples) * np.sum(predictions - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear = np.dot(X, self.weights) + self.bias
        y_pred = self._sigmoid(linear)
        return (y_pred > 0.5).astype(int)

    def predict_proba(self, X):
        linear = np.dot(X, self.weights) + self.bias
        return self._sigmoid(linear)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def compute_loss(self, X, y_true):
        y_pred = self.predict_proba(X)
        # Binary Cross-Entropy Loss
        loss = -np.mean(y_true * np.log(y_pred + 1e-15) + (1-y_true) * np.log(1-y_pred + 1e-15))
        return loss


In [8]:
def logistic_regression_me():
    model = MyLogisticRegression(learning_rate=0.1, n_iters=1000)
    model.fit(trainInputs, trainOutputs)

    trainPred = model.predict(trainInputs)
    testPred = model.predict(testInputs)
    print(f'Accuracy - Train: {accuracy_score(trainOutputs, trainPred):.4f}')
    print(f'Accuracy - Test: {accuracy_score(testOutputs, testPred):.4f}')

    # Predictie pentru (18, 10)
    new_sample = scaler.transform([[18, 10]])
    prob = model.predict_proba(new_sample)
    print(f'Probabilitate malign pentru (18,10): {prob[0]:.4f} => {"Malign" if prob > 0.5 else "Benign"}')


logistic_regression_me()

Accuracy - Train: 0.8901
Accuracy - Test: 0.9035
Probabilitate malign pentru (18,10): 0.6957 => Malign


🏵️ Cerinte opționale

Rezolvarea unei probleme de regresie/clasificare prin:

- folosirea validarii încrucișate

In [9]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(LogisticRegression(), inputs, outputs, cv=5, scoring='accuracy')
print(f"Acuratete medie CV (sklearn): {np.mean(cv_scores):.2f} ± {np.std(cv_scores):.2f}")

Acuratete medie CV (sklearn): 0.89 ± 0.03


- investigarea diferitelor funcții de loss

In [10]:
model = MyLogisticRegression()
model.fit(trainInputs, trainOutputs)
print(f"Loss pe setul de antrenare: {model.compute_loss(trainInputs, trainOutputs):.4f}")

Loss pe setul de antrenare: 0.3268


Ce se întîmplă în cazul clasificarii binare daca se modifică pragul de decizie din 0.5 în alte valori. Cum se poate aprecia calitatea clasificatorului pentru diferite valori ale pragului?

In [11]:
def evaluate_thresholds(model, X, y_true, thresholds=np.linspace(0.1, 0.9, 9)):
    y_proba = model.predict_proba(X)
    results = []

    for thresh in thresholds:
        y_pred = (y_proba > thresh).astype(int)
        acc = accuracy_score(y_true, y_pred)
        results.append((thresh, acc))

    return pd.DataFrame(results, columns=['Threshold', 'Accuracy'])


model = MyLogisticRegression(learning_rate=0.1, n_iters=1000)
model.fit(trainInputs, trainOutputs)
threshold_results = evaluate_thresholds(model, testInputs, testOutputs)
print(threshold_results)

   Threshold  Accuracy
0        0.1  0.771930
1        0.2  0.868421
2        0.3  0.894737
3        0.4  0.912281
4        0.5  0.903509
5        0.6  0.903509
6        0.7  0.894737
7        0.8  0.894737
8        0.9  0.815789
